# Python Bioinformatics Notebook

In [ ]:

# To be run in terminal, not notebook:
# conda create -n bioinfo python=3.11
# conda activate bioinfo
# conda install -c conda-forge biopython scikit-bio numpy pandas matplotlib jupyter
# pip install PyVCF


## Check environment

In [ ]:
import Bio
import skbio
import numpy
import pandas
import matplotlib
import vcfpy

print("All Python packages successfully installed.")

## Section 1: Bioinformatics Data Formats

In bioinformatics, data comes in specialized file formats designed to handle biological sequences, annotations, genomic alignments, variant information, and gene expression data. Here are the most important formats:



- **FASTA Format**: Stores nucleotide or protein sequences

In [ ]:
>gene1_human
ATGGCGTACGCTAGCTAGCTA
>gene2_mouse
ATGCTAGCTAGCTAGTGACTG

In [ ]:
#count of sequences
from Bio import SeqIO
for record in SeqIO.parse("../data/fasta_example.fasta", "fasta"):
    print(record.id, record.seq)


- **FASTQ Format**: Stores sequencing reads and their quality scores from sequencing machines.


In [ ]:
@SEQ_ID_1
GATCTGACTGACTG
+
IIIIIIIIIIIIIH
@SEQ_ID_2
ATCGATCGTAGCTA
+
IIIIIIIHHHHHHG

In [ ]:

from Bio import SeqIO
for record in SeqIO.parse("../data/fastq_example.fastq", "fastq"):
    print(record.id, record.seq, record.letter_annotations["phred_quality"])


- **GenBank Format**: Stores sequences and their annotations, such as gene locations and organism information.

In [ ]:
LOCUS       SCU49845     25 bp    DNA             PLN       21-JUN-1999
DEFINITION  Example GenBank entry.
ACCESSION   SCU49845
VERSION     SCU49845.1
KEYWORDS    .
SOURCE      Artificial Sequence
  ORGANISM  Artificial Sequence
            .
FEATURES             Location/Qualifiers
     gene            1..10
                     /gene="example_gene"
ORIGIN
        1 atggcgtaaa tagctagcta ctagc
//

In [ ]:

from Bio import SeqIO
record = SeqIO.read("../data/gb_example.gb", "genbank")
print(record.annotations)
for feature in record.features:
    print(feature.type, feature.location)


- **GFF/GTF/BED Formats**: Define genomic feature locations, like gene start and end positions.

In [ ]:
chr1    1000    5000    Gene1
chr2    7000    9000    Gene2
chr3    10000	11000    Gene2

In [ ]:
import pandas as pd

bed = pd.read_csv("../data/bed_example.bed", sep="\t", header=None,
                  names=["chrom", "start", "end", "name"])
print(bed)


- **SAM/BAM Formats**: Contain alignments of read data to reference genomes, including mapping scores and alignments.

In [ ]:
@SQ SN:chr1 LN:10000
seq1    0   chr1    1000    255 10M *   0   0   ACGTAGCTAG  *
seq2    0   chr1    1020    255 10M *   0   0   ACGTAGCTAC  *

In [ ]:
!samtools view -S -b ../data/sam_example.sam > ../data/bam_example.bam

In [ ]:
!samtools index ../data/bam_example.bam

In [ ]:
# have bam, convert it to bam
import pysam
bamfile = pysam.AlignmentFile("../data/bam_example.bam", "rb")
for read in bamfile.fetch("chr1", 1000, 2000):
    print(read.query_name, read.query_sequence)

- **VCF Format**: Records genomic variants with reference, alternate alleles, and quality scores.

In [ ]:
##fileformat=VCFv4.2
##source=ExampleSource
#CHROM POS     ID  REF ALT QUAL FILTER INFO
chr1   10176   .   A   AC  50   PASS   DP=20
chr1   10352   .   T   TA  60   PASS   DP=25
chr1   10616   .   C   G   40   q10    DP=10

In [ ]:

import vcfpy

# Open the VCF file
reader = vcfpy.Reader.from_path("../data/vcf_example.vcf")

# Iterate through records
for record in reader:
    print(record.CHROM, record.POS, record.REF, [str(alt) for alt in record.ALT])

- **Gene Expression Format**: Stores expression values in matrix format (RNA-seq counts).

In [ ]:
gene    sample1 sample2 sample3
gene1   100     150     200
gene2   300     250     400
gene3   500     450     600

In [ ]:
import pandas as pd
expr = pd.read_csv("../data/geneexpression_example.txt", sep="\s+", index_col=0)
print(expr.head())

print("Mean expression per sample:")
print(expr.mean())


## Section 2: Bioinformatics Libraries

### Biopython

#### Example A: Parsing FASTA files & computing GC content
- Purpose: Load sequence data from FASTA files and analyze.

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction

for record in SeqIO.parse("../data/fasta_example.fasta", "fasta"):
    gc_content = gc_fraction(record.seq) * 100
    print(f"{record.id}: {gc_content:.2f}% GC")


In [ ]:
# have bam, convert it to bam
import pysam
bamfile = pysam.AlignmentFile("../data/bam_example.bam", "rb")
for read in bamfile.fetch("chr1", 1000, 2000):
    print(read.query_name, read.query_sequence)

In [ ]:

import vcfpy

reader = vcfpy.Reader.from_path("../data/vcf_example.vcf")
for record in reader:
    alt_alleles = [alt.value for alt in record.ALT]  # extract alt allele strings
    print(record.CHROM, record.POS, record.REF, alt_alleles)


#### Example B: Sequence translation and complement
- Purpose: Find Complement, Reverse Complement and Translated Protein

In [ ]:

from Bio.Seq import Seq

sequence = Seq("ATGGCGTACGCTAGCTAGCTA")
print("Original:", sequence)
print("Complement:", sequence.complement())
print("Reverse Complement:", sequence.reverse_complement())
print("Translated Protein:", sequence.translate())


### scikit-bio

#### Example: Pairwise Sequence Alignment
- Purpose: Align two sequences to identify similarities.

In [ ]:

from skbio import DNA, read
from skbio.alignment import local_pairwise_align_ssw

# Read two sequences from the FASTA file
sequences = list(read("../data/fasta_example.fasta", format="fasta", constructor=DNA))

# Assign the sequences
seq1 = sequences[0]
seq2 = sequences[1]

# Run alignment
alignment, score, _ = local_pairwise_align_ssw(seq1, seq2)

# Output
print("Alignment Score:", score)
print(alignment)


### vcfpy

#### Example: Parsing and filtering VCF files
- Purpose: Handle and filter genomic variant data.

In [ ]:

import vcfpy

# Open the VCF file using vcfpy
reader = vcfpy.Reader.from_path("../data/vcf_example.vcf")

# Loop through records and filter by QUAL > 40
for record in reader:
    if record.QUAL is not None and record.QUAL > 40:
        alt_alleles = ",".join(alt.value for alt in record.ALT)
        print(f"{record.CHROM}:{record.POS} {record.REF}->{alt_alleles} QUAL:{record.QUAL}")


## Section 3: Practical Tutorials

### Tutorial 1: Sequence Analysis and Alignment (Biopython & scikit-bio)

In [ ]:
from skbio import read, DNA
from skbio.alignment import local_pairwise_align_ssw
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ---- Step 1: Load sequences from FASTA ----
fasta_file = "../data/fasta_example.fasta"
sequences = list(read(fasta_file, format="fasta", constructor=DNA))
names = [seq.metadata['id'] for seq in sequences]

# Print first two for reference
print("\n----Step 1: Parse sequences ----")
print(f"Seq1 ({names[0]}): {sequences[0]}")
print(f"Seq2 ({names[1]}): {sequences[1]}")

# ---- Step 2: Perform all pairwise alignments ----
print("\n----Step 2: Perform all pairwise alignments ----")
score_matrix = pd.DataFrame(index=names, columns=names, dtype=int)
alignment_details = {}

for i, seq1 in enumerate(sequences):
    for j, seq2 in enumerate(sequences):
        if i <= j:
            alignment, score, _ = local_pairwise_align_ssw(seq1, seq2)
            score_matrix.iloc[i, j] = score
            score_matrix.iloc[j, i] = score  # symmetric
            if i == 0 and j == 1:
                # Save alignment of first pair for display
                alignment_details = {
                    "score": score,
                    "alignment": alignment
                }

# ---- Step 3: Display score matrix ----
print("\n----Step 3: Pairwise Alignment Score Matrix ----")
print(score_matrix)

# ---- Step 4: Show example alignment (Seq1 vs Seq2) ----
print("\n----Step 4: Example Alignment (First Two Sequences) ----")
print(f"Alignment Score: {alignment_details['score']}")
for seq in alignment_details['alignment']:
    print(seq)

# ---- Step 5: Plot score matrix as heatmap ----
print("\n----Step 5: Plot score matrix as heatmap ----")
plt.figure(figsize=(8, 6))
plt.imshow(score_matrix.astype(float), cmap="Blues", interpolation="nearest")
plt.colorbar(label="Alignment Score")
plt.xticks(ticks=np.arange(len(names)), labels=names, rotation=45)
plt.yticks(ticks=np.arange(len(names)), labels=names)
plt.title("Pairwise Alignment Score Heatmap")
plt.tight_layout()
plt.show()


## Section 4: Visualization and Interactive Analysis

### Example A: Sequence Alignment Visualization (matplotlib)
Purpose: Visualize simple alignment scores as a bar plot.

In [ ]:
from skbio import read, DNA
from skbio.alignment import local_pairwise_align_ssw
import matplotlib.pyplot as plt
import itertools

# ---- Step 1: Read sequences from FASTA ----
fasta_file = "../data/fasta_example.fasta"  # Update if needed
sequences = list(read(fasta_file, format="fasta", constructor=DNA))
names = [seq.metadata['id'] for seq in sequences]

# ---- Step 2: Perform pairwise alignments ----
pairs = list(itertools.combinations(enumerate(sequences), 2))  # all unique pairs
labels = []
scores = []

for (i, seq1), (j, seq2) in pairs:
    alignment, score, _ = local_pairwise_align_ssw(seq1, seq2)
    label = f"{names[i]} vs {names[j]}"
    labels.append(label)
    scores.append(score)

# ---- Step 3: Plot ----
plt.figure(figsize=(8, 5))
plt.bar(labels, scores, color="skyblue", edgecolor="black")
plt.ylabel("Alignment Score")
plt.title("Sequence Alignment Scores")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Example B: Variant Quality Distribution (seaborn)
Purpose: Visualize variant quality scores from a VCF file.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import vcfpy

# Open and parse the VCF file
reader = vcfpy.Reader.from_path("../data/vcf_example.vcf")
quals = [record.QUAL for record in reader if record.QUAL is not None]

# Plot the distribution of quality scores
sns.histplot(quals, kde=True)
plt.xlabel('Variant Quality Score')
plt.title('Distribution of Variant Quality Scores')
plt.show()

### Example C: Interactive Volcano Plot (plotly)
Purpose: Interactively visualize RNA-seq results.


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# Load gene expression results
df = pd.read_csv("../data/example_gene_expression.csv")

# Compute -log10 p-values
df['neg_log10_pval'] = -np.log10(df['pvalue'])

# Define significance based on padj and log2FC thresholds
df['significance'] = 'Not Significant'
df.loc[(df['padj'] < 0.05) & (df['log2FoldChange'] > 1), 'significance'] = 'Upregulated'
df.loc[(df['padj'] < 0.05) & (df['log2FoldChange'] < -1), 'significance'] = 'Downregulated'

# Volcano plot
fig = px.scatter(df,
                 x='log2FoldChange',
                 y='neg_log10_pval',
                 color='significance',
                 hover_name='gene',
                 title='Interactive Volcano Plot',
                 labels={
                     'log2FoldChange': 'log2(Fold Change)',
                     'neg_log10_pval': '-log10(p-value)',
                     'significance': 'Regulation Status'
                 },
                 color_discrete_map={
                     'Upregulated': 'red',
                     'Downregulated': 'blue',
                     'Not Significant': 'gray'
                 })

fig.update_traces(marker=dict(size=10, line=dict(width=0.5, color='DarkSlateGrey')))
fig.show()